In [1]:
#dependencies
import pandas as pd
import numpy as np

In [2]:
path_school = 'data/schools_complete.csv' #create variables for filepaths of both csv files
path_student = 'data/students_complete.csv'
school = pd.read_csv(path_school) #create dfs for both csv files
student = pd.read_csv(path_student)

In [3]:
school.head(2) #look at data

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411


In [4]:
student.head(2) #look at data

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61


# District Summary

In [5]:
number_school = school['school_name'].count() #count number of schools
number_student = school['size'].sum() #count number of students
total_budget = school['budget'].sum() #get budget for school district
reading_score_district = round(student['reading_score'].mean(), 6) #get average reading score and round to 6 digits
math_score_district = round(student['math_score'].mean(), 6) #get average math score and round to 6 digits
student['pass_reading'] = student['reading_score'] >= 70 #add column to student df to indicate if they passed reading
student['pass_math'] = student['math_score'] >= 70
student['pass_reading_and_math'] = (student['pass_reading'] == True) & (student['pass_math'] == True)
#calculate percentage of students are passing math, and round to 6 decimals
pass_math_percent = round(student[student['pass_math'] == True].count()['pass_math'] / number_student * 100, 6)
pass_reading_percent = round(student[student['pass_reading'] == True].count()['pass_reading'] / number_student *100, 6)
pass_both = round(student[student['pass_reading_and_math'] == True].count()['pass_reading_and_math'] / number_student * 100, 6)
#create dictionary for df of District Summary
district_dictionary = {'Total Schools': number_school, 'Total Students': number_student, 'Total Budget': total_budget,
                      'Average Math Score': math_score_district, 'Average Reading Score': reading_score_district, 
                      '% Passing Math': pass_math_percent, '% Passing Reading': pass_reading_percent, '% Passing Reading and Math':
                      pass_both}
#pass above dictionary into a DF
district = pd.DataFrame(district_dictionary, index=[0])
district

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Reading and Math
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


# School Summary 



In [7]:
#create groupby variable with average scores, and percentage of students passing math, reading (and both)
school_scores = student.groupby('school_name').mean()[['reading_score', 'math_score', 'pass_math', 
                                                       'pass_reading', 'pass_reading_and_math']]
#create groupby variable with number of students
students_per_school = student.groupby('school_name').count()['student_name']

#convert both groupby variables into dfs
school_scores = pd.DataFrame(school_scores)
students_per_school = pd.DataFrame(students_per_school)

#cleanup column names
students_per_school = students_per_school.rename(columns={'student_name': 'Total Students', 'school_name': 'School Name'})
school_scores = school_scores.rename(columns={'reading_score': 'Average Reading Score', 'math_score': 'Average Math Score',
                                             'pass_math': '% Passing Math', 'pass_reading': '% Passing Reading',
                                             'pass_reading_and_math': '% Passing Reading and Math', 'school_name': 'School Name'})
#convert percent passing reading, math and both to numeric percentages 
school_scores[['% Passing Math', '% Passing Reading', '% Passing Reading and Math']] = school_scores[['% Passing Math', '% Passing Reading', '% Passing Reading and Math']] * 100

#bring school name, type and budget from school df, and cleanup column names
school_summary = school[['school_name', 'type', 'budget']]
school_summary = school_summary.rename(columns={'type': 'School Type', 'budget': 'Total School Budget'})

#merge three dfs together
school_summary = pd.merge(school_summary, school_scores, on='school_name')
school_summary = pd.merge(school_summary, students_per_school, on='school_name')
school_summary = school_summary.rename(columns={'school_name': 'School Name'}) #rename school name column

#calculate per student budget and round to nearest penny
school_summary['Per Student Budget'] = round(school_summary['Total School Budget'] / school_summary['Total Students'], 2)
school_summary.head(15)

,School Name,School Type,Total School Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Reading and Math,Total Students,Per Student Budget
0,Huang High School,District,1910635,81.182722,76.629414,65.683922,81.316421,53.513884,2917,655.0
1,Figueroa High School,District,1884411,81.158020,76.711767,65.988471,80.739234,53.204476,2949,639.0
2,Shelton High School,Charter,1056600,83.725724,83.359455,93.867121,95.854628,89.892107,1761,600.0
3,Hernandez High School,District,3022020,80.934412,77.289752,66.752967,80.862999,53.527508,4635,652.0
4,Griffin High School,Charter,917500,83.816757,83.351499,93.392371,97.138965,90.599455,1468,625.0
5,Wilson High School,Charter,1319574,83.989488,83.274201,93.867718,96.539641,90.582567,2283,578.0
6,Cabrera High School,Charter,1081356,83.975780,83.061895,94.133477,97.039828,91.334769,1858,582.0
7,Bailey High School,District,3124928,81.033963,77.048432,66.680064,81.933280,54.642283,4976,628.0
8,Holden High School,Charter,248087,83.814988,83.803279,92.505855,96.252927,89.227166,427,581.0
9,Pena High School,Charter,585858,84.044699,83.839917,94.594595,95.945946,90.540541,962,609.0


# Top Performing Schools (By Passing Rate)

In [10]:
#sort by schools that have the highest passing rate of both math and reading
school_top_pass = school_summary.sort_values(by='% Passing Reading and Math', ascending=False)
school_top_pass.head(5)

,School Name,School Type,Total School Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Reading and Math,Total Students,Per Student Budget
6,Cabrera High School,Charter,1081356,83.975780,83.061895,94.133477,97.039828,91.334769,1858,582.0
14,Thomas High School,Charter,1043130,83.848930,83.418349,93.272171,97.308869,90.948012,1635,638.0
4,Griffin High School,Charter,917500,83.816757,83.351499,93.392371,97.138965,90.599455,1468,625.0
5,Wilson High School,Charter,1319574,83.989488,83.274201,93.867718,96.539641,90.582567,2283,578.0
9,Pena High School,Charter,585858,84.044699,83.839917,94.594595,95.945946,90.540541,962,609.0


# Bottom Performing Schools (By Passing Rate)

In [11]:
school_bottom_pass = school_summary.sort_values(by='% Passing Reading and Math')
school_bottom_pass.head(5)

,School Name,School Type,Total School Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Reading and Math,Total Students,Per Student Budget
11,Rodriguez High School,District,2547363,80.744686,76.842711,66.366592,80.220055,52.988247,3999,637.0
1,Figueroa High School,District,1884411,81.158020,76.711767,65.988471,80.739234,53.204476,2949,639.0
0,Huang High School,District,1910635,81.182722,76.629414,65.683922,81.316421,53.513884,2917,655.0
3,Hernandez High School,District,3022020,80.934412,77.289752,66.752967,80.862999,53.527508,4635,652.0
12,Johnson High School,District,3094650,80.966394,77.072464,66.057551,81.222432,53.539172,4761,650.0
